In [1]:
import numpy as np
import math
import matplotlib.pyplot as plt

## 3.2 Finite difference method

In [17]:
sigma = 0.3
K = 110
S = 120
r = 0.04
N = 100
T = 1
M1 = 8
M2 = -5

In [18]:
delx = (M1 - M2) / N
deltau = T / N

In [19]:
grid = np.zeros((N,N))


In [20]:
A = np.zeros((N,N))

In [21]:
amin1 = (0.5*sigma**2 * deltau / delx**2)    -    ((r-0.5*sigma**2) *deltau / (2*delx))
aplus1 = (0.5*sigma**2 * deltau / delx**2)    +    ((r-0.5*sigma**2) *deltau / (2*delx))
anow = 1-(sigma**2 * deltau / delx**2) - r * deltau

np.fill_diagonal(A[1:], amin1)
np.fill_diagonal(A[:,1:], aplus1)
np.fill_diagonal(A, anow)


k1 = math.e**M1
k2 = math.e**M1
k3 = math.e**M2
k4 = math.e**M2

A[0,0] = k1
A[0,1] = k2
A[-1,-2] = k3
A[-1, -1] = k4

In [22]:
stocks = np.linspace(M1,M2,N) 
i=0
for row in range(grid.shape[0]):
    ans = math.e**(-r*T) * stocks[row] - math.log(K)
    if ans > 0:
        grid[row,0] = ans
    else:
        grid[row,0] = 0

In [23]:
for column in range(N-2):
    grid[:,column+1] = np.linalg.solve(A, grid[:,column])

In [24]:
grid[:,0]

array([2.98583515, 2.85967088, 2.73350661, 2.60734234, 2.48117807,
       2.3550138 , 2.22884953, 2.10268526, 1.97652099, 1.85035672,
       1.72419245, 1.59802818, 1.47186391, 1.34569964, 1.21953537,
       1.0933711 , 0.96720683, 0.84104256, 0.71487829, 0.58871402,
       0.46254975, 0.33638548, 0.21022121, 0.08405694, 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.     

In [25]:
np.matmul(A, grid[:,0])

array([1.74252079e+04, 2.85857553e+00, 2.73246173e+00, 2.60634793e+00,
       2.48023412e+00, 2.35412032e+00, 2.22800651e+00, 2.10189271e+00,
       1.97577891e+00, 1.84966510e+00, 1.72355130e+00, 1.59743749e+00,
       1.47132369e+00, 1.34520989e+00, 1.21909608e+00, 1.09298228e+00,
       9.66868473e-01, 8.40754669e-01, 7.14640865e-01, 5.88527061e-01,
       4.62413256e-01, 3.36299452e-01, 2.10185648e-01, 8.51849477e-02,
       2.25436740e-03, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
      

In [26]:
grid[:,-1]

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])